In [ ]:
!pip3 install pandas_ods_reader
from pandas_ods_reader import read_ods

In [ ]:
data = read_ods("hi_3500.ods",1, headers=False)
data.head()

,column.0,column.1
0,लोग वतन तक खा जाते हैं इसका इसे यकीन नहींमान ज...,negative
1,गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...,negative
2,ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है,negative
3,यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...,negative
4,अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...,negative


In [ ]:
data.columns = ['text','labels']
data.head()

,text,labels
0,लोग वतन तक खा जाते हैं इसका इसे यकीन नहींमान ज...,negative
1,गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...,negative
2,ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है,negative
3,यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...,negative
4,अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...,negative


In [ ]:
data.describe(), data.labels.unique()

(                                                     text    labels
 count                                                9077      9077
 unique                                               7832         3
 top     ससुर ने किया बलात्कार ISLAMIC LAW-NOW WIFE IS ...  positive
 freq                                                    8      3254,
 array(['negative', 'positive', 'neutral'], dtype=object))

In [ ]:
def convert(label):
  if label=='negative':
    return -1
  if label=='positive':
    return 1
  else:
    return 0

In [ ]:
labels_numerical = [convert(t) for t in data.labels]

In [ ]:
labels_numerical[:4]

[-1, -1, -1, -1]

In [ ]:
def preprocess(text):
  text = text.split()
  text = [t for t in text if not t.isalpha()]
  return " ".join(text)

In [ ]:
text = [preprocess(t) for t in data.text]

In [ ]:
text[:4]

['लोग खा जाते हैं इसका इसे यकीन नहींमान जाएगा तू ले जाकर दिल्ली इसे दिखा ला दोस्त',
 'गुमनाम है मिटने वाले लोग आतन्कवादियों से मिलकर मशहूर हो गये',
 'ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है',
 'यूपी में बड़े स्तर दंगे करवा सकती है बीजेपी: मुलायम सिंह यादव -']

In [ ]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

xtrain,xtest,ytrain,ytest = train_test_split(text,labels_numerical,test_size = 0.3,stratify = labels_numerical)
len(xtrain),len(xtest)

(6353, 2724)

In [ ]:
'''cv = CountVectorizer().fit(xtrain+xtest)
transformed_xtrain = cv.fit_transform(xtrain)
transformed_xtest = cv.transform(xtest)'''
tfidf = TfidfVectorizer().fit(xtrain+xtest)
transformed_xtrain = tfidf.fit_transform(xtrain)
transformed_xtest = tfidf.transform(xtest)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression().fit(transformed_xtrain,ytrain)
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
rfc = RandomForestClassifier().fit(transformed_xtrain,ytrain)
gbc = GradientBoostingClassifier().fit(transformed_xtrain,ytrain)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
from sklearn.metrics import classification_report,accuracy_score
lr_preds = lr.predict(transformed_xtest)
rfc_preds = rfc.predict(transformed_xtest)
gbc_preds = gbc.predict(transformed_xtest)
#print(classification_report(lr_preds,ytest))
#print(classification_report(rfc_preds,ytest))
#print(classification_report(gbc_preds,ytest))

In [ ]:
import statistics
res = []
exceptions = 0
for i in range(len(lr_preds)):
  try:
    res.append(statistics.mode([lr_preds[i],rfc_preds[i],gbc_preds[i]]))
  except:
    exceptions+=1
    res.append(rfc_preds[i])

In [ ]:
import numpy as np
print(classification_report(np.array(res),ytest)),print(classification_report(rfc_preds,ytest))

              precision    recall  f1-score   support

          -1       0.70      0.74      0.72       900
           0       0.90      0.84      0.87       851
           1       0.70      0.71      0.71       973

    accuracy                           0.76      2724
   macro avg       0.77      0.76      0.76      2724
weighted avg       0.76      0.76      0.76      2724

              precision    recall  f1-score   support

          -1       0.74      0.73      0.74       967
           0       0.90      0.85      0.87       840
           1       0.69      0.73      0.71       917

    accuracy                           0.77      2724
   macro avg       0.78      0.77      0.77      2724
weighted avg       0.77      0.77      0.77      2724



(None, None)

In [ ]:
def ensemble(text):
  text = tfidf.transform(text)
  try:
    result = statistics.mode([lr.predict(text),rfc.predict(text),gbc.predict(text)])
  except:
    result = rfc.predict(text)
  if result==0:
    return "neutral"
  if result==1:
    return "positive"
  return "negative"

In [ ]:
ensemble(["कुछ ही देर में फैक्ट्री में आग की लपटों के साथ काले धुएं का गुबार उठता देखकर श्रमिकों में भगदड़ मच गई।"])

'negative'

In [ ]:
ensemble(["मैं खुश हूँ"])

'positive'

In [ ]:
ensemble(["RT @aajtak: दिल्ली: शाहदरा के पूर्व विधायक जितेन्द्र सिंह शंटी ने अदालत के फैसले का स्वागत किया है जिसमें कोर्ट ने कांग्रेस के सीनियर नेता…"])

'neutral'